```{contents}
```
## **Tool Policies in LangGraph**

**Tool Policies** in LangGraph define **rules and constraints** that govern *when*, *how*, and *whether* agents and nodes are allowed to call tools.
They are a critical **production-safety mechanism** for building reliable, auditable, and secure LLM systems.

---

### **1. Why Tool Policies Exist**

Unrestricted tool access causes:

| Risk                  | Consequence         |
| --------------------- | ------------------- |
| Cost explosion        | Unlimited API calls |
| Security failures     | Unsafe commands     |
| Hallucinated actions  | Wrong tool usage    |
| Compliance violations | No audit trail      |
| System damage         | Destructive actions |

Tool policies convert LLMs from **free-form reasoners** into **controlled operators**.

---

### **2. Core Functions of Tool Policies**

Tool policies enforce:

1. **Authorization** — Who can use which tool
2. **Validation** — Whether inputs are acceptable
3. **Constraints** — How often and how much a tool can be used
4. **Routing** — Which tool is selected
5. **Safety gates** — When human approval is required
6. **Observability** — What is logged and audited

---

### **3. Policy Placement in LangGraph**

Tool policies operate at three layers:

| Layer       | Purpose                     |
| ----------- | --------------------------- |
| Node level  | Restrict a specific node    |
| Agent level | Restrict agent capabilities |
| Graph level | Global enforcement          |

---

### **4. Tool Policy Types**

| Policy Type           | Enforces                  |
| --------------------- | ------------------------- |
| Allowlist             | Which tools are permitted |
| Denylist              | Which tools are forbidden |
| Rate policy           | Call frequency limits     |
| Cost policy           | Budget ceilings           |
| Input policy          | Parameter validation      |
| Output policy         | Result sanitization       |
| Human-approval policy | Manual confirmation       |
| Security policy       | Access & sandboxing       |
| Compliance policy     | Logging & retention       |
| Escalation policy     | Trigger review path       |

---

### **5. Simple Tool Policy Example**

```python
from langgraph.prebuilt import ToolNode

allowed_tools = ["search", "calculator"]

tool_node = ToolNode(
    tools=tools,
    tool_policy={
        "allow": allowed_tools,
        "max_calls": 3,
        "require_approval": False
    }
)
```

---

### **6. Cost & Rate Enforcement Example**

```python
tool_policy = {
    "allow": ["web_search"],
    "max_calls": 2,
    "max_cost": 0.05
}
```

---

### **7. Human Approval Gate Example**

```python
tool_policy = {
    "allow": ["email_sender"],
    "require_approval": True
}
```

Graph behavior:

```
Agent → Approval Node → Tool Execution → Continue
```

---

### **8. Policy-Controlled Tool Routing**

```python
def tool_router(state):
    if state["risk"] == "high":
        return "human_review"
    return "safe_tool"
```

---

### **9. Production-Grade Tool Policy Architecture**

```
LLM Agent
   ↓
Policy Engine
   ↓
Tool Authorization Layer
   ↓
Tool Sandbox
   ↓
Execution + Audit Log
```

---

### **10. Benefits of Tool Policies**

| Benefit      | Result                       |
| ------------ | ---------------------------- |
| Safety       | Prevents destructive actions |
| Cost control | Budget enforcement           |
| Compliance   | Regulatory alignment         |
| Auditability | Complete traceability        |
| Reliability  | Deterministic behavior       |

---

### **11. Mental Model**

> **LLM decides intent → Tool policy decides permission**

Tool policies are the **governor** that makes LangGraph safe for production.


### Demonstration

In [6]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langchain_core.tools import tool

# ----------------------------
# 1. Define State
# ----------------------------

class State(TypedDict):
    query: str
    tool_name: str
    tool_result: str
    approved: bool
    calls: int

# ----------------------------
# 2. Define Tools
# ----------------------------

@tool
def web_search(q: str) -> str:
    """Web Search"""
    return f"Search result for: {q}"

@tool
def send_email(msg: str) -> str:
    """Send Mail"""
    return "Email sent"

TOOLS = {
    "web_search": web_search,
    "send_email": send_email
}

# ----------------------------
# 3. Tool Policy (implemented manually)
# ----------------------------

ALLOWED_TOOLS = {"web_search"}
MAX_CALLS = 2

def enforce_policy(state: State):
    if state["tool_name"] not in ALLOWED_TOOLS:
        raise ValueError("Tool not allowed")
    if state["calls"] >= MAX_CALLS:
        return {"tool_result": "Rate limit exceeded"}

# ----------------------------
# 4. Nodes
# ----------------------------

def agent(state: State):
    return {
        "tool_name": "web_search",
        "calls": state["calls"] + 1
    }

def approval(state: State):
    print("Human approval required → approved")
    return {"approved": True}

def tool_executor(state: State):
    enforce_policy(state)
    tool = TOOLS[state["tool_name"]]
    result = tool.invoke(state["query"])
    return {"tool_result": result}

def router(state: State):
    if not state["approved"]:
        return "approval"
    if state["calls"] >= MAX_CALLS:
        return END
    return "tool"

# ----------------------------
# 5. Build Graph
# ----------------------------

builder = StateGraph(State)

builder.add_node("agent", agent)
builder.add_node("approval", approval)
builder.add_node("tool", tool_executor)

builder.set_entry_point("agent")

builder.add_edge("agent", "approval")

builder.add_conditional_edges("approval", router, {
    "tool": "tool",
    END: END
})

builder.add_edge("tool", "agent")

graph = builder.compile()

# ----------------------------
# 6. Run
# ----------------------------

result = graph.invoke({
    "query": "LangGraph",
    "tool_name": "",
    "tool_result": "",
    "approved": False,
    "calls": 0
})

print("\nFinal State:", result)

Human approval required → approved
Human approval required → approved

Final State: {'query': 'LangGraph', 'tool_name': 'web_search', 'tool_result': 'Search result for: LangGraph', 'approved': True, 'calls': 2}
